In [ ]:
import os
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from natsort import natsorted

In [ ]:
# Set the path to the directory containing the image stacks
stack_dir = 'C:/Users/au519867/OneDrive - Aarhus universitet/Skrivebord/Raw_Images/RAW_Stacks/Threshold'

# Get a list of all TIFF files in the input directory
files = [f for f in os.listdir(stack_dir) if f.endswith('.tiff')]
print(files)

green_stack = tifffile.imread(os.path.join(stack_dir, files[0]))
red_stack = tifffile.imread(os.path.join(stack_dir, files[1]))

#Get length of image stacks
num_images_green = len(green_stack)
num_images_red = len(red_stack)
if num_images_green == num_images_red:
    print("Equal stacks")
    num_images = num_images_green
else:
    print("unequal stacks")

print("stack=",num_images)

In [ ]:
# Loop through all red/green pairs of images in the stacks an get the RED/GREEN intensity ratio
red_green_ratio = []
for i in range(num_images):

    # Calculate the intensity ratio between the two images
    ratio = red_stack[i] / green_stack[i]
    red_green_ratio.append(ratio)


In [ ]:
# Define number of sections for each image to calculate mean ratios
num_sections = 3

# Calculate the height of each section
section_height = green_stack.shape[1] // num_sections

# Loop over eack image in the ratio stack and divide it into sections
ratio_section_means = []
ratio_sections = []
for image in red_green_ratio:
    sections = [image[i*section_height:(i+1)*section_height, :] for i in range(num_sections)]
    ratio_sections.append(sections)

    # Calculate the mean ratio of eacjh section and append it to the list
    ratio_section_mean = [np.nanmean(section) for section in sections]
    ratio_section_means.append(ratio_section_mean)

# Reshape array of mean ratios to 
ratio_section_means = np.reshape(ratio_section_means, (-1, num_sections))
print(ratio_section_means)

In [ ]:
# Define the image index to show
image_idx = 0

# Get an example image from the green stack and its sections to see how its divided
image = green_stack[image_idx]
sections = ratio_sections[image_idx]

for section_idx in range(num_sections):
    section = sections[section_idx]
    plt.imshow(section, cmap="gray")
    plt.title(f'Image {image_idx}, Section {section_idx}')
    plt.show()


In [ ]:
# Calculate the mean of means for each image and the standard deviation
ratio_mean_of_means = []
ratio_std_of_means =[]

for i in range(len(ratio_section_means)):
    mean_ratio = np.nanmean(ratio_section_means[i])
    std_ratio = np.nanstd(ratio_section_means[i])
    ratio_mean_of_means.append(mean_ratio)
    ratio_std_of_means.append(std_ratio)

print(ratio_mean_of_means)
print(ratio_std_of_means)

In [ ]:
# Set the path to the directory containing the image filenames
files_dir = 'C:/Users/au519867/OneDrive - Aarhus universitet/Skrivebord/Raw_Images'
files = [f for f in os.listdir(files_dir) if f.endswith('.tiff')]

# Sort filenames to match with stack index
files = natsorted(files)

# Initialize an empty dataframe
df_ratios = pd.DataFrame(columns=['Air Saturation', 'Intensity Ratio', 'Std'])

# Add filenames and corresponding red/green ratio to the dataframe
for i in range(num_images):
    ratio_df = ratio_mean_of_means[i]
    std_df = ratio_std_of_means[i]
    filename = os.path.splitext(files[i])[0][22:]  # get the filename without extension
    df_ratios.loc[i] = [filename, ratio_df, std_df]

df_ratios['Intensity Ratio'] = df_ratios['Intensity Ratio'].round(2)

# Print the dataframe
print(df_ratios)

In [ ]:
# Create Scatterplot with errorbars
plt.errorbar(x=df_ratios['Air Saturation'], y=df_ratios['Intensity Ratio'], yerr=df_ratios['Std'], fmt='o')

# Set X-axis limits
#plt.ylim(ymin=1, ymax=2.5)
plt.show()

In [ ]:
 #Save the DataFrame as a CSV file
df_ratios.to_csv('C:/Users/au519867/OneDrive - Aarhus universitet/Skrivebord/Raw_Images/RAW_Stacks/Intensity_Ratio/ratio_data.csv',header=True, index=False, decimal=',')